In [2]:

# coding: utf-8

# In[1]:


import json
import pickle
import random
import numpy as np

def clean_review(review,word_set,max_size):
    """
    Removes characters from the review string that are in the to_filter string
    The output is a filtered, lower case representation of the review, splitted to words by space
    """
    for c in TO_SPACE:
        review = review.replace(c,' ')
    review = review.split(' ')[0:max_size]

    review_stripped = []
    for word in review:
        to_remove = False
        for c in FILTERED_CHARS:
            if c in word:
                to_remove = True
                break
        if to_remove == False:
            new_word = word.strip()
            if len(new_word)>0:
                review_stripped.append(new_word)
    return review_stripped

def get_glove_dict(file):
    """
    Load glove dictionary file. 
    Glove is a pre-trained nlp embedding layer from https://nlp.stanford.edu/projects/glove/
    """
    with open(file,encoding="utf8") as glove:
        return {l[0]: np.asarray(l[1:], dtype="float32") for l in [line.split() for line in glove]}

DIR = "../data/"
WORK_DIR = "data_cnn/"
OUTPUT_DIR = WORK_DIR + "tcenr_data/"#WORK_DIR + "amazon_movies/"
FILE_NAME = DIR + "yelp_filtered.json"#DIR+"reviews_Movies_and_TV_5.json"#"Electronics_5.json"#"yelp_review_restraunts.json"
GLOVE_FILE = DIR + "glove.6B."
REVIEW_MAX_SIZE = 500
# Glove file sizeW
WORD_EMBEDDING_FILE = 50
# Characters to filter out of reviews
FILTERED_CHARS = '[!"#$%&()*+-./:;<=>?@[\\]^_`{|}~]'
TO_SPACE = '[.,\t\n]'

glove_to_load = GLOVE_FILE + str(WORD_EMBEDDING_FILE) + "d.txt"
glove_dictionary = get_glove_dict(glove_to_load)

raw_words = {}
for word in glove_dictionary.keys():
    raw_words[word] = None


# In[2]:


LIMIT = 0
reviews= []
scores = []
users = []
pois = []
user_counts = {}
poi_counts = {}
word_dict = {}
word_dict_rev = {}
reviews_file = open(FILE_NAME,encoding="utf8")
data_loaded = 0
word_count = 1

for line in reviews_file:
    data_loaded+=1
    if (data_loaded == LIMIT):
        break
    if (data_loaded % 10000 == 0):
        print("Processing record {}".format(data_loaded))
    record = json.loads(line)
    score = record["stars"]
    review = record["text"]
    user = record["user_id"]
    poi = record["business_id"]
    #user = record["reviewerID"]
    #poi = record["asin"]
    #score = int(record["overall"])
    #review = record["reviewText"]
    for c in TO_SPACE:
        review = review.replace(c,' ')
    review = review.split(' ')[0:REVIEW_MAX_SIZE]
    
    new_review = []
    
    for word in review:
        to_remove = False
        word = word.strip().lower()
        for c in FILTERED_CHARS:
            if c in word:
                to_remove = True
                break
        word = word.replace('\'','')
        if to_remove == False and word in raw_words:
            
            if word not in word_dict:
                word_dict[word] = word_count
                word_dict_rev[word_count] = word
                word_count+=1
            new_review.append(word_dict[word])
            
    
    if (len(new_review)>0):
        reviews.append(new_review)
        scores.append(score)
        users.append(user)
        if user not in user_counts:
            user_counts[user] = 1
        else:
            user_counts[user] += 1
        pois.append(poi)
        if poi not in poi_counts:
            poi_counts[poi] = 1
        else:
            poi_counts[poi] += 1
        
reviews_file.close()


# In[3]:


new_glove = {}
new_glove[0] = np.array([0.0] * WORD_EMBEDDING_FILE)
for word in word_dict:
    word_idx = word_dict[word]
    new_glove[word_idx] = glove_dictionary[word]

Processing record 10000
Processing record 20000
Processing record 30000
Processing record 40000
Processing record 50000
Processing record 60000
Processing record 70000
Processing record 80000
Processing record 90000
Processing record 100000
Processing record 110000
Processing record 120000
Processing record 130000
Processing record 140000


In [3]:
train_reviews = []
train_scores = []
test_reviews = []
test_scores = []
for i in range(len(reviews)):
    user = users[i]
    poi = pois[i]
    review = reviews[i]
    if user_counts[user]>=10 and poi_counts[poi]>=10:
        set_choice = random.random()
        if (set_choice>0.1):
            train_reviews.append(review)
            train_scores.append(scores[i])
        else:
            test_reviews.append(review)
            test_scores.append(scores[i])
train_set = {"reviews": train_reviews, "scores": train_scores}
test_set = {"reviews": test_reviews, "scores": test_scores}

In [4]:
print(len(reviews), len(train_reviews)+len(test_reviews))

141019 139228


In [5]:
print("Train size:{}, Test size:{}".format(len(train_reviews),len(test_scores)))
print("dictionary size:{}".format(len(word_dict)))

with open(OUTPUT_DIR + "glove_file.pkl",'wb') as file:
    pickle.dump(list(new_glove.values()),file)
    
with open(OUTPUT_DIR + "word_dictionary.pkl",'wb') as file:
    pickle.dump(word_dict,file)
    
with open(OUTPUT_DIR + "word_dictionary_reverse.pkl",'wb') as file:
    pickle.dump(word_dict_rev,file)
    
with open(OUTPUT_DIR + "train.pkl",'wb') as file:
    pickle.dump(train_set,file)
    
with open(OUTPUT_DIR + "test.pkl",'wb') as file:
    pickle.dump(test_set,file)



Train size:125376, Test size:13852
dictionary size:58487


In [6]:
len(new_glove)

58488

In [11]:
OUTPUT_DIR = WORK_DIR + "amazon/"